<font size=6 color='#4a86e8'><b><i>Analyst Exercise 2</i></b></font><p align="right">Calculate scroll rates for the top 10 articles</p>

<i><font size=4 color='#4a86e8'>Import packages</font></i>

In [ ]:
import pandas as pd
import re

<i><font size=4 color='#4a86e8'>First let's define a helper function that will clean the data<br>(this function standardizes column names, inserts a numeric Percentile column and checks for duplicated PagePaths)</font></i>

In [ ]:
def prep_data(df, target):
    # If the target already exists, cancel since data prep already done
    if 'Target' in df.head():
        print('Data prep already completed')
        return df
    
    # If the target is Pageviews, the data source is GA, and the pageviews need to be 'unsampled'
    if target == 'Pageviews':
        print('WARNING: "unsampling" target column')
        df['Pageviews'] = df.EventAction.apply(lambda x: 10 if x != 'Scroll 90%' else 1) * df['Pageviews']
    
    # Add a numeric Percentile column
    if 'Percentile' not in df.head():
        df.insert(2, 'Percentile', [int(re.sub('[^0-9]', '', s))/100 for s in df['EventAction']])
    
    # Rename the target column to 'Target' to standardize
    df = df.rename(columns={target: 'Target'})
    new_names = ', '.join(df.head())
    print(f'New column names: {new_names}')
    
    # Lowercase all of the PagePaths then check for duplicates
    df = df.assign(PagePath = [d.lower() for d in df['PagePath']])
    dups = df[df.duplicated(subset=['PagePath', 'EventAction'], keep=False)].sort_values(by=['PagePath', 'EventAction'])
    
    # If there are duplicates offer the option to clean them up
    if len(dups) > 0:
        if input('Duplicates detected, would you like to view them (y/n)?') == 'y':
            print(dups)
        
        if input('Would you like to consolidate duplicates (y/n)?') == 'y':
            deduped = dups.groupby(['PagePath', 'EventAction', 'Percentile']).sum().reset_index()
            
            orig_len = len(df)
            
            df = df.drop_duplicates(subset=['PagePath', 'EventAction'], keep=False).append(deduped).reset_index(drop=True)
            
            print(f'{orig_len - len(df)} rows removed')
            
            if input('Would you like to see the result of de-duplication (y/n)?') == 'y':
                print(df)
                
    return df

<i><font size=4 color='#4a86e8'>Load csv data and preview</font></i>

In [ ]:
dat = pd.read_csv('~/AnalystExercises/Data/scroll_depth.csv') # Define the source data here
dat

<i><font size=4 color='#4a86e8'>Prep the data using the helper function above.<br>(the function will standardize column names, add a numeric Percentile column check for clean duplicates)</font></i>

In [ ]:
dat = prep_data(dat, 'Pageviews')

<i><font size=4 color='#4a86e8'>Define a function to calculate the average scroll rate for each subset of the dataframe
<br>Sum of (# of views per each percentile * the percentile)/total pageviews</font></i>

In [ ]:
def avg_scroll_rate(x):
    return round(sum([(row['Target'] * row['Percentile']) for index, row in x.iterrows()])/sum(x['Target']), 3)

<i><b><font size=4 color='#4a86e8'>Finally, we can group our page paths, apply the scroll rate calculation, and view the results</i></b></font>

In [ ]:
grouped = dat.groupby('PagePath')

avg_scrolls = grouped.apply(avg_scroll_rate).sort_values(ascending=False).rename('AvgScrollRateGA').to_frame()

dat_agg = dat.filter(['PagePath', 'Target']).groupby('PagePath').sum().rename(columns={'Target': 'TargetGA'})

results = dat_agg.merge(avg_scrolls, on='PagePath', how='left').sort_values(by='AvgScrollRateGA', ascending=False)

results

<b><font size=4 color='#4a86e8'>Let's compare the results above with those from BigQuery</b></font>

In [ ]:
dat_bq = pd.read_csv('~/AnalystExercises/Data/scroll_depth_bq.csv') # Define the source data here

dat_bq = prep_data(dat_bq, 'UnsampledEventCount')

grouped_bq = dat_bq.groupby('PagePath')

avg_scrolls_bq = grouped_bq.apply(avg_scroll_rate).sort_values(ascending=False).rename('AvgScrollRateBQ').to_frame()

dat_agg_bq = dat_bq.filter(['PagePath', 'Target']).groupby('PagePath').sum().rename(columns={'Target': 'TargetBQ'})

results_bq = dat_agg_bq.merge(avg_scrolls_bq, on='PagePath', how='left').sort_values(by='AvgScrollRateBQ', ascending=False)

results_comp = results.merge(results_bq, on='PagePath', how='left')

for c in ['Target', 'AvgScrollRate']:
    results_comp[f'{c} Err (%)'] = round(100 * (results_comp[f'{c}BQ'] - results_comp[f'{c}GA']) / results_comp[f'{c}GA'], 2)

results_comp = results_comp.reindex(columns=['TargetGA', 'TargetBQ', 'Target Err (%)', 'AvgScrollRateGA', 'AvgScrollRateBQ', 'AvgScrollRate Err (%)'])
    
results_comp